2021.01 ~ 2022.10 멜론 월간 차트 탑 100 크롤링

In [57]:
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [82]:
## 1. 멜론 월간 차트 탑 100 ------------------------------------------------------------------

# 정보 저장할 리스트
info = []

# 크롬 드라이버 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
url = 'https://www.melon.com/chart/month/index.htm'
driver.get(url)

# 월 변수
month = 10

# while문 끝내기 위한 변수
w_break = 0

# 멜론 월별 탑 100 정보 추출
while True:
    sleep(3)
    # 년도 넘기기
    if month == 0:
        month = 12
        driver.find_element(By.CSS_SELECTOR, f'#conts > div.calendar_prid > div > button').click()
        driver.find_element(By.CSS_SELECTOR, f'#conts > div.calendar_prid > div > div > dl > dt > button.btn_round.small.pre').click()
        driver.find_element(By.CSS_SELECTOR, f'#conts > div.calendar_prid > div > div > dl > dd.month_calendar > ul > li:nth-child({month}) > a > span').click()
        w_break += 1
        if w_break == 2:
            break
    else:
        driver.find_element(By.CSS_SELECTOR, f'#conts > div.calendar_prid > div > button').click()
    sleep(3)
    try:
        driver.find_element(By.CSS_SELECTOR, f'#conts > div.calendar_prid > div > div > dl > dd.month_calendar > ul > li:nth-child({month}) > a > span').click()
        sleep(3)
    except:
        continue
    for i in range(1,101):
        rank = driver.find_element(By.CSS_SELECTOR, f'tbody > tr:nth-child({i}) > td:nth-child(2) > div > span.rank').text
        driver.find_element(By.CSS_SELECTOR, f'tbody > tr:nth-child({i}) > td:nth-child(5) > div > a').click()
        sleep(3)
        title = driver.find_element(By.CSS_SELECTOR, f'#downloadfrm > div > div > div.entry > div.info > div.song_name').text
        name = driver.find_element(By.CSS_SELECTOR, f'#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)').text
        date = driver.find_element(By.CSS_SELECTOR, f'#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
        genre = driver.find_element(By.CSS_SELECTOR, f'#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
        driver.back()
        info.append([rank,title,name, genre,date])
    month -= 1
driver.quit()

# 차트 정보 저장
chart_table = pd.DataFrame(info, columns=['순위','제목','가수','장르', '발매일'])
chart_table.to_csv(f'C:\\Users\\min\\melon_rank.csv', index=False)

C:\Users\min\AppData\Local\Temp/ipykernel_10132/671279928.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
C:\Users\min\AppData\Local\Temp/ipykernel_10132/671279928.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)


In [39]:
# 2. 차트에 포함 되어있는 가수의 추가 정보 추출 ----------------------------------------------

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")

# 크롬 드라이버 설정
driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
url = 'https://www.melon.com/index.htm'
driver.get(url)

## 가수 소속사, 활동 유형 추출 EX) (여자)아이들, 큐브엔터테이먼트, 여자 그룹
s_info = []
for name in melon_chart['가수'].unique():
    query = driver.find_element(By.CSS_SELECTOR,'#top_search')
    query.send_keys(name)
    query.send_keys(Keys.ENTER)
    sleep(1)
    ## 검색해도 나오지 않는 경우 처리
    try:
        category = driver.find_element(By.CSS_SELECTOR, f'#conts > div.section_atist > div.wrap_cntt.clfix.d_artist_list > div.atist_dtl_info > dl > dd:nth-child(4)').text
    except:
        category, company = '', ''
        s_info.append([name, category, company])
        driver.find_element(By.CSS_SELECTOR,'#top_search').clear()
        continue
    driver.find_element(By.CSS_SELECTOR, f'#conts > div.section_atist > div.wrap_cntt.clfix.d_artist_list > div.atist_dtl_info > div > a').click()
    sleep(1)
    # 소속사가 명시 되지 않은 경우 처리
    try:
        company = driver.find_element(By.XPATH, f'//dt[contains(text(), "소속사")]/following-sibling::dd').text
    except:
        company = ''
    s_info.append([name, category, company])
driver.quit()

## 가수 정보 저장
singer_table = pd.DataFrame(s_info,columns = ['가수', '활동유형', '소속사'])
singer_table.to_csv(f'C:\\Users\\min\\singer_table.csv', index=False)

C:\Users\min\AppData\Local\Temp/ipykernel_13600/29250580.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
C:\Users\min\AppData\Local\Temp/ipykernel_13600/29250580.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)


In [77]:
# 3. 소속사의 모회사 추출 ----------------------------------------------------------------------
company_table = pd.DataFrame(singer_table['소속사'].unique(),columns=['소속사'])

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
url = 'https://www.google.com/'
driver.get(url)

sleep(2)

## 소속사의 상위 회사 정보 추출
com_info = []
for com in singer_table['소속사'].unique():
    query = driver.find_element(By.CSS_SELECTOR,'div.a4bIc > input')
    query.send_keys(com)
    query.send_keys(Keys.ENTER)
    sleep(2)
    ## 상위 회사의 정보가 소유자, 모회자 두가지 경우로 명시
    try:
        company = driver.find_element(By.XPATH, f'//a[@class="fl"][contains(text(), "소유자")]/following::a[@class="fl"]').text
    except:
        try:
            company = driver.find_element(By.XPATH, f'//a[@class="fl"][contains(text(), "모회사")]/following::a[@class="fl"]').text
        except:
            company = com
    com_info.append(company)
    driver.find_element(By.CSS_SELECTOR,'div.a4bIc > input').clear()
driver.quit()

## 모회사 정보 저장
company_table['모회사'] = com_info
company_table.to_csv(f'C:\\Users\\min\\company_table.csv', index=False)